In [2]:
import tkinter as tk
import json
json_decoder = json.JSONDecoder()

settings = []

In [3]:
def read_settings(file_name = "./config/settings.json"):
    with open(file_name, mode="r", encoding="utf-8") as input_file:
        return json_decoder.decode(input_file.read())
settings = read_settings();

In [4]:
def edit_person(person_id,render_hook):
    print(person_id)
    id = int(person_id)
    edit_window = tk.Tk()
    person = {}
    edit_window.title("Add Person")

    if "owners" in settings and len(settings["owners"])>id:
        person = settings["owners"][id]
        edit_window.title("Edit "+person["pid"])

    # Create a new frame `frm_form` to contain the Label
    # and Entry widgets for entering address information
    edit_frm_form = tk.Frame(master=edit_window,relief=tk.SUNKEN, borderwidth=3)
    # Pack the frame into the window
    edit_frm_form.pack()

    # List of field labels
    labels = [
        ("first_name","First Name:"),
        ("last_name", "Last Name:"),
        ("email","Email:"),
        ("pid","PID:"),
    ]

    widget_set = {"test":None}

    # Loop over the list of field labels
    for idx, text in enumerate(labels):
        # Create a Label widget with the text from the labels list
        label = tk.Label(master=edit_frm_form, text=text[1])
        # Create an Entry widget
        entry = tk.Entry(master=edit_frm_form, width=50)
        if text[0] in person:
            entry.insert(tk.END, person[text[0]])
        widget_set[text[0]] = entry
        # Use the grid geometry manager to place the Label and
        # Entry widgets in the row whose index is idx
        label.grid(row=idx, column=0, sticky="e")
        entry.grid(row=idx, column=1)

    def get_current(id,render_hook):
        person = {}
        if "owners" in settings and len(settings["owners"])>id:
            person = settings["owners"][id]
        for val in widget_set:
            if not val == "test":
                label = widget_set[val]
                person[val] = label.get()
        if "owners" in settings and not len(settings["owners"])>id:
            settings["owners"].append(person)
        render_hook()
        return person

    # Create a new frame `frm_buttons` to contain the
    # Submit and Clear buttons. This frame fills the
    # whole window in the horizontal direction and has
    # 5 pixels of horizontal and vertical padding.
    frm_buttons = tk.Frame(master=edit_window)
    frm_buttons.pack(fill=tk.X, ipadx=5, ipady=5)
    def process_end():
        get_current(int(person_id),render_hook)
        close_app(master=edit_window)


    # Create the "Submit" button and pack it to the
    # right side of `frm_buttons`
    btn_submit = tk.Button(master=frm_buttons, text="Ok", command=process_end)
    btn_submit.pack(side=tk.RIGHT, padx=10, ipadx=10)

    # Create the "Clear" button and pack it to the
    # right side of `frm_buttons`
    btn_clear = tk.Button(master=frm_buttons, text="Clear")
    btn_clear.pack(side=tk.RIGHT, ipadx=10)

    # Start the application
    edit_window.mainloop()
def delete_person(id,master):
    settings["owners"].pop(int(id))
    parent = master.master
    master.destroy()
    render_people(settings["owners"], parent)

In [5]:
def get_edit_row(pid,person, master, render_hook):
    label = tk.Label(master=master, text=person["first_name"]+" "+person["last_name"])
    label.grid(row=pid, column=0)
    lol = str(pid)
    btn_open = tk.Button(master=master,text="Edit", command=lambda:edit_person(lol,render_hook=render_hook))
    btn_open.grid(row=pid, column=1)
    btn_open = tk.Button(master=master,text="Delete", command=lambda:delete_person(lol,master))
    btn_open.grid(row=pid, column=2)

In [7]:
def close_app(master):
    master.destroy()

def render_people(owners,frm_form):
    print("hello")
    people = tk.Frame(master=frm_form, relief=tk.SUNKEN, borderwidth=3)
    for pid, person in enumerate(owners):
            get_edit_row(pid,person,people,render_hook=lambda:render_people(owners,frm_form))
    btn_open = tk.Button(master=people,text="Add New", command=lambda:edit_person(len(owners),render_hook=lambda:render_people(owners,frm_form)))
    btn_open.grid(row=len(owners),column=2, sticky="ew")
    people.grid(row=1,column=1, sticky="ew")

def save_settings():
    with open("./config/test.json", mode="w", encoding="utf-8") as output_file:
        text = json.JSONEncoder().encode(settings)
        output_file.write(text)
def change_setting(field):
    value = settings[field]
    if value:
        value = False
    else:
        value = True
    settings[field] = value

def edit_settings():
    window = tk.Tk()
    window.title("Settings")
    frm_form = tk.Frame(relief=tk.SUNKEN, borderwidth=3)
    # Pack the frame into the window
    frm_form.pack()

    labels = [("field","type", "text"),
              ("owners", "array", "Owners:"),
              ("data","file","Data Path:"),
              ("match_locations", "checkbox", "Match Locations:")]

    for idx, box in enumerate(labels):
        if box[1] == "type":
            continue
        if box[1] == "array":
            if box[0] in settings:
                label = tk.Label(master=frm_form, text=box[2])
                label.grid(row=idx, column=0, sticky="w")
                render_people(settings[box[0]],frm_form)
            continue
        if box[1] == "checkbox":
            label = tk.Label(master=frm_form, text=box[2])
            label.grid(row=idx, column=0, sticky="w")
            check = tk.Checkbutton(master=frm_form,
                command=lambda: change_setting(box[0]), variable=box[2],
                onvalue='1', offvalue='0')
            check.grid(row=idx,column=1, sticky="w")
            continue

        # Create a Label widget with the text from the labels list
        label = tk.Label(master=frm_form, text=box[2])
        # Create an Entry widget
        entry = tk.Entry(master=frm_form, width=50)
        if box[0] in settings:
            entry.insert(tk.END, settings[box[0]])
        # Use the grid geometry manager to place the Label and
        # Entry widgets in the row whose index is idx
        label.grid(row=idx, column=0, sticky="w")
        entry.grid(row=idx, column=1)

    frm_buttons = tk.Frame(master=window)
    frm_buttons.pack(fill=tk.X, ipadx=5, ipady=5)
    btn_submit = tk.Button(master=frm_buttons, text="Save", command=lambda:save_settings())
    btn_submit.pack(side=tk.RIGHT, padx=10, ipadx=10)

    # Create the "Clear" button and pack it to the
    # right side of `frm_buttons`
    btn_clear = tk.Button(master=frm_buttons, text="Exit", command=lambda: close_app(master=window))
    btn_clear.pack(side=tk.RIGHT, ipadx=10)

    window.mainloop()
edit_settings()

hello


In [70]:
def read_settings(file_name = "./config/settings.json"):
    with open(file_name, mode="r", encoding="utf-8") as input_file:
        settings = json_decoder.decode(input_file.read())
read_settings();